In [9]:
import pandas as pd
import prepare
from prepare import prep_equip_df

In [10]:
def get_hwyrail():
    df = pd.read_csv('hwy_rail_accidents_2012.csv', index_col=False)
    df1 = pd.read_csv('hwy_rail_accidents_2013.csv', index_col=False)  
    df2 = pd.read_csv('hwy_rail_accidents_2014.csv', index_col=False)  
    df3 = pd.read_csv('hwy_rail_accidents_2015.csv', index_col=False)  
    df4 = pd.read_csv('hwy_rail_accidents_2016.csv', index_col=False)  
    df5 = pd.read_csv('hwy_rail_accidents_2017.csv', index_col=False)  
    df6 = pd.read_csv('hwy_rail_accidents_2018.csv', index_col=False)  
    df7 = pd.read_csv('hwy_rail_accidents_2019.csv', index_col=False)  
    df8 = pd.read_csv('hwy_rail_accidents_2020.csv', index_col=False) 
    big_df = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8])
    return big_df

In [11]:
hwy_df = get_hwyrail()
hwy_df

,AMTRAK,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,...,NARR1,NARR2,NARR3,NARR4,NARR5,SUBDIV,ROADCOND,VIDEOT,VIDEOU,Unnamed: 103
0,NaN,12,3,UP,0312RS009,NaN,NaN,,,12.0,...,HIEHWAY USER'S ACTIONS: STOPPED ON CROSSING ( ...,NaN,NaN,NaN,NaN,WOODBRIDGE,B,1,2,NaN
1,NaN,12,3,UP,0312RS025,NaN,NaN,,,12.0,...,HIGHWAY USER'S ACTIONS: STOPPED ON CROSSING ( ...,D LEFT THE SCENE.,NaN,NaN,NaN,FRESNO SUB,A,1,2,NaN
2,NaN,12,10,BNSF,CA1012201,NaN,NaN,,,12.0,...,DRIVER AGE UNKNOW. 41: ABANDONED VEHICLE,NaN,NaN,NaN,NaN,SAN BERNARDINO,A,1,2,NaN
3,NaN,12,10,BNSF,CA1012202,NaN,NaN,,,12.0,...,12/18/12 PER CONTRA COSTA COUNTY CORONERS REPO...,NaN,NaN,NaN,NaN,STOCKTON,A,1,1,NaN
4,NaN,12,7,BNSF,CA0712201,NaN,NaN,,,12.0,...,NaN,NaN,NaN,NaN,NaN,STOCKTON,A,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,NaN,20,2,SFRV,2132020,NaN,NaN,,,20.0,...,TRAIN P645 WAS TRAVELLING SOUTHBOUND AND STRUC...,ALLY INJURED; AGE IS CURRENTLY UNKNOWN.,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,2,2,NaN
1887,NaN,20,1,SFRV,1052020,NaN,NaN,,,20.0,...,TWO MALES ON A BICYCLE RAN UNDER CROSSING GATE...,HE BIKE WAS HIT AND KILLED WHILE THE OTHER ONE...,PORT IS PENDING SO AGE OF DECEASED IS STILL UN...,NaN,NaN,SOUTHFLRAILCORRIDOR,A,1,1,NaN
1888,NaN,20,1,SFRV,1092020,NaN,NaN,,,20.0,...,TRAIN P647 STRUCK A VEHICLE THAT STOPPED ON TH...,CT. HE WAS TRAVELING EAST IN THE WESTBOUND TRA...,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,1,1,NaN
1889,NaN,20,1,SFRV,1172020,NaN,NaN,,,20.0,...,TRAIN P628 STRUCK A VEHICLE THAT STOPPED ON TH...,UNINJURED.,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,2,2,NaN


In [12]:
hwy_df.describe().T

,count,mean,std,min,25%,50%,75%,max
IYR,18995.0,15.995051,2.548730,12.0,14.0,16.0,18.0,20.0
IMO,18995.0,6.598052,3.531982,1.0,3.0,7.0,10.0,12.0
IYR2,19.0,17.947368,1.682382,14.0,17.0,18.0,19.5,20.0
IMO2,19.0,7.157895,4.003653,1.0,3.0,8.0,11.0,12.0
IYR3,18879.0,16.006038,2.550739,12.0,14.0,16.0,18.0,20.0
IMO3,18879.0,6.596801,3.531095,1.0,3.0,7.0,10.0,12.0
DUMMY1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASINJRR,18995.0,0.417531,1.278679,0.0,0.0,0.0,1.0,87.0
YEAR,18995.0,15.995051,2.548730,12.0,14.0,16.0,18.0,20.0
MONTH,18995.0,6.598052,3.531982,1.0,3.0,7.0,10.0,12.0


In [13]:
def get_equiprail():
    
    '''This function acquires the equipment rail accident data, concats all of the dataframes into one'''
    
    
    df = pd.read_csv('rail_equip_accidents_2012.csv', index_col=False)
    df1 = pd.read_csv('rail_equip_accidents_2013.csv', index_col=False)  
    df2 = pd.read_csv('rail_equip_accidents_2014.csv', index_col=False)  
    df3 = pd.read_csv('rail_equip_accidents_2015.csv', index_col=False)  
    df4 = pd.read_csv('rail_equip_accidents_2016.csv', index_col=False)  
    df5 = pd.read_csv('rail_equip_accidents_2017.csv', index_col=False)  
    df6 = pd.read_csv('rail_equip_accidents_2018.csv', index_col=False)  
    df7 = pd.read_csv('rail_equip_accidents_2019.csv', index_col=False)  
    df8 = pd.read_csv('rail_equip_accidents_2020.csv', index_col=False) 
    
    equip_df = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8])
    
    return equip_df

In [14]:
equip_df = get_equiprail()

In [15]:
equip_df

,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,IMO3,...,RCL,Latitude,Longitud,SIGNAL,MOPERA,ADJUNCT1,ADJUNCT2,ADJUNCT3,SUBDIV,Unnamed: 145
0,12,5,BNSF,GC0512102,NaN,NaN,NaN,NaN,12,5,...,0.0,29.755900,-95.293100,1,1,G,NaN,NaN,NORTH SHORE,NaN
1,12,5,BNSF,GC0512105,NaN,NaN,NaN,NaN,12,5,...,0.0,29.500571,-95.597089,2,5,Z,NaN,NaN,GALVESTON,NaN
2,12,2,NS,98074,12.0,2.0,CSX,100906,12,2,...,NaN,34.612981,-86.988133,1,1,B,NaN,NaN,ALABAMA,NaN
3,12,5,BNSF,GC0512106,NaN,NaN,NaN,NaN,12,5,...,0.0,31.446576,-96.207606,2,5,Z,NaN,NaN,HOUSTON,NaN
4,12,5,BNSF,GC0512112,NaN,NaN,NaN,NaN,12,5,...,0.0,29.772155,-95.384013,2,5,Z,NaN,NaN,HOUSTON,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,20,2,NS,137017,NaN,NaN,NaN,NaN,20,2,...,0.0,37.271098,-79.997944,2,5,K,NaN,NaN,POCAHONTAS,NaN
2136,20,5,BNSF,PR0520107,NaN,NaN,NaN,NaN,20,5,...,0.0,44.216404,-105.293504,1,1,Q,NaN,NaN,ORIN,NaN
2137,20,5,NECR,NEC420520D,NaN,NaN,NaN,NaN,20,5,...,0.0,44.481200,-73.101600,2,2,NaN,NaN,NaN,ROXBURY,NaN
2138,20,1,CSX,188190,NaN,NaN,NaN,NaN,20,1,...,3.0,31.213524,-82.353588,2,5,NaN,NaN,NaN,THOMASVILLE,NaN


In [16]:
equip_df = prep_equip_df(equip_df)
equip_df

,iyr,imo,railroad,type,state,temp,visiblty,weather,trnspd,typspd,...,otherinj,county,cntycd,passtrn,narr1,narr2,latitude,longitud,signal,date
0,12,5,BNSF,1,48,84,2,1,10,E,...,0,HARRIS,201.0,N,Y-HOU38810-04 DERAILED 1 CAR WHILE DOUBLING FR...,PSI BRAKE PIPE REDUCTION AT 8 MPH WHILE ADDING...,29.755900,-95.293100,1,2012-05-05 08:00:00
1,12,5,BNSF,1,48,70,4,3,5,E,...,0,FORT BEND,157.0,N,C-RWMSLP1-29 DERAILED IN SMITHERS LAKE COAL PL...,NaN,29.500571,-95.597089,2,2012-05-11 02:25:00
2,12,2,NS,1,1,39,2,3,0,NaN,...,0,MORGAN,103.0,NaN,CSX TRAIN. CSXT TRAIN Q235.18 PULLING SOUTH AT...,"ADS, 0 EMPTIES AND 3768 TONS, IN A #10 TURNOUT...",34.612981,-86.988133,1,2012-02-19 11:30:00
3,12,5,BNSF,1,48,84,2,1,3,E,...,0,FREESTONE,161.0,N,C-NAMDOL0-02 WAS CLEARED TO DEPART PLANT BUT H...,", RESULTING IN DERAILMENT.",31.446576,-96.207606,2,2012-05-16 15:40:00
4,12,5,BNSF,1,48,70,4,1,5,R,...,0,HARRIS,201.0,N,Y-DYT28310-30 PULLING FROM 908 TRACK DERAILED ...,"ICAL ISSUES CAUSE DERAILMENT, BUT BNSF UNABLE ...",29.772155,-95.384013,2,2012-05-31 03:36:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,20,2,NS,1,51,39,4,2,6,E,...,0,ROANOKE,161.0,N,TRAIN UR22 SHOVING EAST UP THE NORTH LADDER IN...,ARS.,37.271098,-79.997944,2,2020-02-01 23:20:00
2136,20,5,BNSF,6,56,45,2,1,0,E,...,0,CAMPBELL,5.0,N,MAINTENANCE OF WAY UNDERCUTTER HAD RAILCARS UN...,AIN 2 TRACK DUE TO IMPROPER OPERATION OF TRAIN...,44.216404,-105.293504,1,2020-05-11 15:00:00
2137,20,5,NECR,1,50,35,4,1,22,R,...,0,CHITTENDEN,7.0,N,TRAIN 324 TRAVELING SOUTHBOUND ON THE NECR MAI...,"REST ON THEIR SIDE, AND 4 LOADED LUMBER FLATS...",44.481200,-73.101600,2,2020-05-13 22:00:00
2138,20,1,CSX,1,13,55,2,2,7,E,...,0,WARE,299.0,N,Y19231 WAS SHOVING A CUT IN F11 WHEN THE RAIL ...,NaN,31.213524,-82.353588,2,2020-01-31 09:56:00


In [17]:
equip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22402 entries, 0 to 2139
Data columns (total 52 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   iyr       22402 non-null  int64         
 1   imo       22402 non-null  int64         
 2   railroad  22402 non-null  object        
 3   type      22402 non-null  int64         
 4   state     22402 non-null  int64         
 5   temp      22402 non-null  int64         
 6   visiblty  22402 non-null  int64         
 7   weather   22402 non-null  int64         
 8   trnspd    22402 non-null  int64         
 9   typspd    20946 non-null  object        
 10  trndir    21232 non-null  float64       
 11  tons      22402 non-null  int64         
 12  typeq     20944 non-null  object        
 13  typtrk    22402 non-null  int64         
 14  headend1  22402 non-null  int64         
 15  loadf1    22402 non-null  int64         
 16  loadp1    22402 non-null  int64         
 17  emptyf1   224